# Data notebook

## Setup

In [ ]:
!python -m pip install tqdm requests nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from IPython.display import display
import pandas as pd
import requests
import re
from urllib.request import urlopen
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
#from fa2 import ForceAtlas2
from scipy import stats
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import io
from tqdm import tqdm
from heapq import nlargest 
import seaborn as sns
sns.set()

## Getting the data

Based on https://www.dr.dk/nyheder/politik/folketingsvalg/valgte a list of the top 3 politicians by vote from each party with at least 5 mandates as of the 2022 danish election is created. 

Data is collected using the <font color="red"> TODO </font> twitter API. 

As the twitter API needs the twitter user id to collect tweets, the twitter user ids are manually collected using the ["Find Twitter ID"](https://www.codeofaninja.com/tools/find-twitter-id/) by [CodeOfaNinja](https://www.codeofaninja.com/).

The resulting politicians and their twitter ids are seen in the dataframe below.

We observe that we have 12 different parties and 36 politicians in total.

In [ ]:
from Data.twitter_ids import twitter_ids

twitter_id_df = pd.DataFrame(columns=['name', "party", 'twitter_id'])
i = 0
for party in twitter_ids:
    for person in twitter_ids[party]:
        twitter_id_df.loc[i, :] = [person, party, twitter_ids[party][person]]
        i += 1

display(twitter_id_df)

,name,party,twitter_id
0,magnus_heunicke,socialdemokratiet,22695562
1,nicolai_wammen,socialdemokratiet,2803948786
2,mattias_tesfaye,socialdemokratiet,546254893
3,jakob_ellemann,venstre,155584627
4,soren_gade,venstre,975064362359623680
5,sophie_lohde,venstre,44611200
6,lars_lokke,moderaterne,26201346
7,henrik_frandsen,moderaterne,1249019841924734977
8,rosa_eriksen,moderaterne,1560192117858861056
9,jacob_mark,sf,2373406198


The dataset is then created with the Twitter API using these ids and the code used for this is collected in the file `data_request.ipynb`. <font color="red"> TODO </font> collect data retrieving code in py file and explain it here.

This results in multiple csv files for each politician looking like this:

In [ ]:
print("Example of data frame: ")
example_df = pd.read_csv("Data/alternativet/christina_olumeko_0.csv", index_col=0)
display(example_df.head())

print("\033[1mExample of tweet: \033[0m")
print(example_df.loc[0, "text"])

Example of data frame: 


,edit_history_tweet_ids,id,text
0,['1566066637933088769'],1566066637933088769,"Rigtig ærgerligt, at Socialdemokratiet dropper..."
1,['1566055972031922176'],1566055972031922176,“Kommissionen for den glemte kvindekamp” under...
2,['1565978593318129665'],1565978593318129665,@Mhvid @SophieHAndersen Ja og særligt når komm...
3,['1565956735612985347'],1565956735612985347,Grineren video fra ⁦@Vejdirektoratet⁩. Lad os ...
4,['1565947952501334018'],1565947952501334018,@AFreltoft @politiken Dejligt at Københavns go...


Example of tweet: 
Rigtig ærgerligt, at Socialdemokratiet dropper at arbejde for mindre gadeparkering i København🚗Det optager nemlig meget plads, og er samtidig nødvendigt for at nå Københavns klimamål. Alternativet arbejder videre 🌱 #dkgreen https://t.co/lUB0dMk5ud


### Data cleaning

As we are only interested in the text of each tweet, we will collect the text for each politician as a list of strings (tweets) and add it as a new column to the data. However, we would like to clean the text a bit before doing this, for this purpose we do the following:

- only keep words and numbers, i.e. no emojis
- remove punctuation, stopwords and urls
- make all words lowercase

<font color="red"> Maybe we can use mrjob to do this? </font>

### Single tweet

In [ ]:
t = example_df.loc[0, "text"]

wnl = nltk.WordNetLemmatizer() 
STOPWORDS = nltk.corpus.stopwords.words('danish') #+ ["http"]

def clean_tweet(tweet):
    tokensraw = word_tokenize(tweet) #get tokens 
    tokens = [word.lower() for word in tokensraw if word.isalnum()] #only get words and numbers
    tokens = [wnl.lemmatize(t) for t in tokens] #lemmatize tokens
    tokens = [w for w in tokens if w.lower() not in STOPWORDS] #remove stopwords
    return tokens

print("Original tweet: \n", t)
print("\n")
print("Clean tweet: \n" + " ".join(clean_tweet(t)))


Original tweet: 
 Rigtig ærgerligt, at Socialdemokratiet dropper at arbejde for mindre gadeparkering i København🚗Det optager nemlig meget plads, og er samtidig nødvendigt for at nå Københavns klimamål. Alternativet arbejder videre 🌱 #dkgreen https://t.co/lUB0dMk5ud


Clean tweet: 
rigtig ærgerligt socialdemokratiet dropper arbejde mindre gadeparkering optager nemlig plads samtidig nødvendigt nå københavns klimamål alternativet arbejder videre dkgreen http


### Multiple tweets

In [ ]:
import glob
# All files and directories ending with .txt and that don't begin with a dot:
print(glob.glob("Data/*")) 


['Data/__pycache__', 'Data/twitter_ids.py', 'Data/dansk_folkeparti', 'Data/enhedslisten', 'Data/test_payload.csv', 'Data/sf', 'Data/alternativet', 'Data/venstre', 'Data/radikale', 'Data/danmarksdemokraterne', 'Data/socialdemokratiet', 'Data/konservative', 'Data/nye_borgerlige', 'Data/backup_plan_tweets.csv', 'Data/moderaterne', 'Data/liberal_alliance']


In [ ]:
#cleaning each twitter period in the party folder:

subfolders = [ f.path for f in os.scandir("Data/") if f.is_dir() ][1:]
for folder in subfolders:
    files = glob.glob(folder + "*.csv")
    tokens_list = []
    for partimedlem in files:
        for i in range(partimedlem.shape[0]):
            raw = partimedlem.text.iloc[i]
            tokensraw = word_tokenize(raw) #get tokens from wikipage
            tokens = [word.lower() for word in tokensraw if word.isalnum()] #only get words and numbers
            tokens = [wnl.lemmatize(t) for t in tokens] #lemmatize tokens
            marvel_tokens += tokens

In [ ]:
wnl = nltk.WordNetLemmatizer() 

tokens_list = []
for i in range(data.shape[0]):
    raw = data["text"].iloc[i]
    tokensraw = word_tokenize(raw) #get tokens 
    tokens = [word.lower() for word in tokensraw if word.isalnum()] #only get words and numbers
    tokens = [wnl.lemmatize(t) for t in tokens] #lemmatize tokens
    tokens_list += tokens

NameError: name 'data' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=30797f9c-952e-45b4-98d4-31c9ac73ae78' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>